# Crypto Altcoin Analysis
This workbook will walk through extracting data from Coinbase on all altcoins available with a USD pairing, then clean data and do some performance and correlation analysis.



In [2]:
# Imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
import csv
import cbpro

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Setup Public Client for Coinbase
public_client = cbpro.PublicClient()

In [26]:
# Pull a list of all currencies (Assets and pairings)
currency_response_list = public_client.get_currencies()
currency_response_list

[{'id': 'ALGO',
  'name': 'Algorand',
  'min_size': '1',
  'status': 'online',
  'message': '',
  'max_precision': '0.000001',
  'convertible_to': [],
  'details': {'type': 'crypto',
   'symbol': 'A',
   'network_confirmations': 1,
   'sort_order': 93,
   'crypto_address_link': 'https://algoexplorer.io/address/{{address}}',
   'crypto_transaction_link': 'https://algoexplorer.io/tx/{{txId}}',
   'push_payment_methods': ['crypto'],
   'group_types': [],
   'display_name': '',
   'processing_time_seconds': 5,
   'min_withdrawal_amount': 0.1,
   'max_withdrawal_amount': 700000}},
 {'id': 'DASH',
  'name': 'Dash',
  'min_size': '1',
  'status': 'online',
  'message': '',
  'max_precision': '0.00000001',
  'convertible_to': [],
  'details': {'type': 'crypto',
   'symbol': '',
   'network_confirmations': 2,
   'sort_order': 47,
   'crypto_address_link': 'https://chain.so/address/DASH/{{address}}',
   'crypto_transaction_link': 'https://chain.so/tx/DASH/{{address}}',
   'push_payment_methods':

In [27]:
# Pull only the assets that are marked as crypto

# Create empty list of cryptos
crypto_list = []

# If the asset is labeled as a crypto add to crypto list
for asset in currency_response_list:
    if asset['details']['type'] == 'crypto':
        crypto_list.append(asset['id'])

print(crypto_list)

['ALGO', 'DASH', 'OXT', 'ATOM', 'KNC', 'MIR', 'REP', 'ICP', 'CGLD', 'COMP', 'NMR', 'BAND', 'XLM', 'EOS', 'ZRX', 'BAT', 'LOOM', 'UNI', 'YFI', 'LRC', 'CVC', 'DNT', 'MANA', 'GNT', 'REN', 'LINK', 'BTC', 'BAL', 'LTC', 'ETH', 'BCH', 'ETC', 'USDC', 'ZEC', 'RLC', 'DAI', 'WBTC', 'NU', 'FIL', 'AAVE', 'SNX', 'BNT', 'GRT', 'SUSHI', 'MLN', 'ANKR', 'CRV', 'STORJ', 'SKL', 'AMP', '1INCH', 'ENJ', 'NKN', 'OGN', 'FORTH', 'GTC', 'TRB', 'XRP', 'XTZ', 'CTSI', 'MKR', 'UMA', 'DOGE', 'ADA', 'USDT', 'DOT', 'CHZ', 'SHIB', 'SOL', 'BOND', 'LPT', 'QNT', 'KEEP', 'CLV', 'MASK', 'MATIC', 'OMG', 'POLY', 'FARM', 'FET', 'PAX', 'RLY']


In [28]:
# Get a list of all Coinbase Products
all_products_list = public_client.get_products()
all_products_list

[{'id': 'BAL-USD',
  'base_currency': 'BAL',
  'quote_currency': 'USD',
  'base_min_size': '0.1',
  'base_max_size': '6700',
  'quote_increment': '0.00001',
  'base_increment': '0.001',
  'display_name': 'BAL/USD',
  'min_market_funds': '1.0',
  'max_market_funds': '100000',
  'margin_enabled': False,
  'fx_stablecoin': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'trading_disabled': False,
  'status': 'online',
  'status_message': ''},
 {'id': 'GRT-EUR',
  'base_currency': 'GRT',
  'quote_currency': 'EUR',
  'base_min_size': '10',
  'base_max_size': '2500000',
  'quote_increment': '0.0001',
  'base_increment': '0.01',
  'display_name': 'GRT/EUR',
  'min_market_funds': '10.0',
  'max_market_funds': '92000',
  'margin_enabled': False,
  'fx_stablecoin': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'trading_disabled': False,
  'status': 'online',
  'status_message': ''},
 {'id': 'MIR-EUR',
  'base_currency': 'MIR',
  'quot

In [29]:
# Find product ids that have a USD quote_currency and create a list

# Create Empty List for USD paired products
usd_product_id_list = []
for product in all_products_list:
    if product['quote_currency'] == 'USD':
        usd_product_id_list.append(product['id'])
print(usd_product_id_list)

['BAL-USD', 'AAVE-USD', 'PAX-USD', 'SKL-USD', 'BCH-USD', 'NMR-USD', 'LTC-USD', 'FET-USD', 'FARM-USD', 'STORJ-USD', 'SUSHI-USD', 'NU-USD', 'ETH-USD', 'COMP-USD', 'ADA-USD', 'KEEP-USD', 'CRV-USD', 'WBTC-USD', 'CLV-USD', 'REN-USD', 'ETC-USD', 'ATOM-USD', '1INCH-USD', 'MKR-USD', 'CTSI-USD', 'DAI-USD', 'ZRX-USD', 'BAT-USD', 'SOL-USD', 'OMG-USD', 'MASK-USD', 'FORTH-USD', 'CGLD-USD', 'DOT-USD', 'ZEC-USD', 'XLM-USD', 'ICP-USD', 'RLY-USD', 'FIL-USD', 'AMP-USD', 'USDT-USD', 'OXT-USD', 'MIR-USD', 'ALGO-USD', 'CHZ-USD', 'BTC-USD', 'ANKR-USD', 'DOGE-USD', 'MANA-USD', 'BNT-USD', 'BAND-USD', 'KNC-USD', 'MATIC-USD', 'UMA-USD', 'NKN-USD', 'LRC-USD', 'SNX-USD', 'XTZ-USD', 'LINK-USD', 'GTC-USD', 'UNI-USD', 'GRT-USD', 'EOS-USD', 'TRB-USD', 'LPT-USD', 'YFI-USD', 'MLN-USD', 'ENJ-USD', 'DASH-USD', 'BOND-USD', 'REP-USD', 'POLY-USD', 'RLC-USD', 'QNT-USD', 'OGN-USD']
